In [1]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [2]:
## 경고 메세지 표시 안함
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
# 라이브러리 탑재
import folium
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.rc('font',family='Malgun Gothic')
plt.rcParams['axes.unicode_minus']=False  # '- 표시

In [4]:
# 승하차 인원 CSV 파일 읽어오기
data = pd.read_csv("data/seoul-metro-2021.logs.csv")

# 데이터 확인
data

,timestamp,station_code,people_in,people_out
0,2021-01-01T05:00:00.000+09:00,150,86,85
1,2021-01-01T06:00:00.000+09:00,150,111,355
2,2021-01-01T07:00:00.000+09:00,150,157,438
3,2021-01-01T08:00:00.000+09:00,150,306,592
4,2021-01-01T09:00:00.000+09:00,150,333,841
...,...,...,...,...
1941548,2021-12-31T19:00:00.000+09:00,2828,74,263
1941549,2021-12-31T20:00:00.000+09:00,2828,73,145
1941550,2021-12-31T21:00:00.000+09:00,2828,95,209
1941551,2021-12-31T22:00:00.000+09:00,2828,54,138


In [5]:
# 지하철 역 정보 CSV 파일 읽어오기
station_info = pd.read_csv("data/seoul-metro-station-info.csv")

# 데이터 확인
station_info

,station.code,station.fr_code,line.num,line.name,line.name_sub,line.station_seq,station.name_full,station.name,station.name_chc,station.name_chn,station.name_en,station.name_jp,geo.latitude,geo.longitude,geo.sigungu_code,geo.sigungu_name,geo.addres_road,geo.address_land,geo.phone
0,158,124,1,1호선,지하철1호선,1,청량리(서울시립대입구),청량리|서울시립대입구,祭基洞,祭基洞,Jegidong,チェギドン,37.580178,127.046835,11060,동대문구,서울특별시 동대문구 왕산로 지하205(전농동),서울특별시 동대문구 전농동 620-69 청량리역(1호선),02-6110-1241
1,157,125,1,1호선,지하철1호선,2,제기동,제기동,新設洞,新设洞,Sinseoldong,シンソルトン,37.578103,127.034893,11060,동대문구,서울특별시 동대문구 왕산로 지하93(제기동),서울특별시 동대문구 제기동 65 제기동역(1호선),02-6110-1251
2,156,126,1,1호선,지하철1호선,3,신설동,신설동,東廟앞,东庙,Dongmyo,トンミョアプ,37.575297,127.025087,11060,동대문구,서울특별시 동대문구 왕산로 지하1(신설동),서울특별시 동대문구 신설동 76-5 신설동역(1호선),02-6110-1261
3,159,127,1,1호선,지하철1호선,4,동묘앞,동묘앞,淸凉里(서울市立大入口),清凉里(首尔市立大学),Cheongnyangni(University of Seoul),チョンニャンニ,37.572627,127.016429,11010,종로구,서울특별시 종로구 종로 359(숭인동),서울특별시 종로구 숭인동 117 동묘앞역(1호선),02-6110-1271
4,155,128,1,1호선,지하철1호선,5,동대문,동대문,東大門,东大门,Dongdaemun,トンデムン,37.571420,127.009745,11010,종로구,서울특별시 종로구 종로 지하302(창신동),서울특별시 종로구 창신동 492-1 동대문역(1호선),02-6110-1281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,2823,823,8,8호선,지하철8호선,14,남한산성입구(성남법원.검찰청),남한산성입구|성남법원|검찰청,南漢山城入口(城南法院檢察廳),南汉山城(城南法院·检察厅),Namhansanseong(Seongnam Court & Prosecutors' O...,ナムハンサンソンイック,37.451535,127.159816,31021,성남시수정구,경기도 성남시 수정구 산성대로 지하445(단대동),경기도 성남시 수정구 단대동 96 남한산성입구역(8호선),02-6311-8231
281,2824,824,8,8호선,지하철8호선,15,단대오거리,단대오거리,丹垈오거리,丹垈五岔路口,Dandaeogeori,タンデオゴリ,37.445210,127.156866,31021,성남시수정구,경기도 성남시 수정구 산성대로 지하365(신흥동),경기도 성남시 수정구 신흥동 2467 단대오거리역(8호선),02-6311-8241
282,2825,825,8,8호선,지하철8호선,16,신흥,신흥,新興,新兴,Sinheung,シンフン,37.440918,127.147564,31021,성남시수정구,경기도 성남시 수정구 산성대로 지하280(신흥동),경기도 성남시 수정구 신흥동 2467 신흥역(8호선),02-6311-8251
283,2826,826,8,8호선,지하철8호선,17,수진,수진,壽進,寿进,Sujin,スジン,37.437428,127.140722,31021,성남시수정구,경기도 성남시 수정구 산성대로 지하200(수진동),경기도 성남시 수정구 수진동 2205-1 수진역(8호선),02-6311-8261


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1941553 entries, 0 to 1941552
Data columns (total 4 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   timestamp     object
 1   station_code  int64 
 2   people_in     int64 
 3   people_out    int64 
dtypes: int64(3), object(1)
memory usage: 59.3+ MB


In [10]:
# 승하차 인원 그룹화하여 더하기
station_sum = data.groupby("station_code").sum()

# 데이터 확인하기
station_sum

,people_in,people_out
station_code,,
150,12697273,12109991
151,5997344,6030491
152,9638952,9284693
153,7704599,7090896
154,6651283,6609055
...,...,...
2824,3206211,3097722
2825,1395919,1495603
2826,1614590,1475010


In [7]:
station_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   station.code       285 non-null    int64  
 1   station.fr_code    285 non-null    object 
 2   line.num           285 non-null    int64  
 3   line.name          285 non-null    object 
 4   line.name_sub      285 non-null    object 
 5   line.station_seq   285 non-null    int64  
 6   station.name_full  285 non-null    object 
 7   station.name       285 non-null    object 
 8   station.name_chc   285 non-null    object 
 9   station.name_chn   285 non-null    object 
 10  station.name_en    285 non-null    object 
 11  station.name_jp    285 non-null    object 
 12  geo.latitude       285 non-null    float64
 13  geo.longitude      285 non-null    float64
 14  geo.sigungu_code   285 non-null    int64  
 15  geo.sigungu_name   285 non-null    object 
 16  geo.addres_road    285 non

In [8]:
station_info = station_info[['station.code','geo.latitude', 'geo.longitude']]
station_info

,station.code,geo.latitude,geo.longitude
0,158,37.580178,127.046835
1,157,37.578103,127.034893
2,156,37.575297,127.025087
3,159,37.572627,127.016429
4,155,37.571420,127.009745
...,...,...,...
280,2823,37.451535,127.159816
281,2824,37.445210,127.156866
282,2825,37.440918,127.147564
283,2826,37.437428,127.140722


In [9]:
# 인덱스 변경
station_info = station_info.set_index('station.code')

# 데이터 확인하기
station_info

,geo.latitude,geo.longitude
station.code,,
158,37.580178,127.046835
157,37.578103,127.034893
156,37.575297,127.025087
159,37.572627,127.016429
155,37.571420,127.009745
...,...,...
2823,37.451535,127.159816
2824,37.445210,127.156866
2825,37.440918,127.147564


In [11]:
# 데이터 합치기
joined_data = station_sum.join(station_info)

# 데이터 확인하기
joined_data

,people_in,people_out,geo.latitude,geo.longitude
station_code,,,,
150,12697273,12109991,37.554648,126.972559
151,5997344,6030491,37.564718,126.977108
152,9638952,9284693,37.570161,126.982923
153,7704599,7090896,37.571607,126.991806
154,6651283,6609055,37.570926,127.001849
...,...,...,...,...
2824,3206211,3097722,37.445210,127.156866
2825,1395919,1495603,37.440918,127.147564
2826,1614590,1475010,37.437428,127.140722


In [ ]:
# joined_data = joined_data[['people_in','people_out','geo.latitude', 'geo.longitude']]
# joined_data

In [12]:
# 서울 지도 만들기
seoul_in = folium.Map(location=[37.55, 126.98], zoom_start = 12)

# 지도 확인하기
seoul_in

In [14]:
# 히트맵 플러그인 모듈 탑재
from folium.plugins import HeatMap 

# 히트맵을 지도에 추가 (승차인원)
HeatMap(data = joined_data[['geo.latitude', 'geo.longitude', 'people_in']]).add_to(seoul_in)

# 지도 확인
seoul_in


In [15]:
seoul_in.save('seoul_in.html')

In [16]:

# 히트맵을 지도에 추가 (하차인원)
HeatMap(data = joined_data[['geo.latitude', 'geo.longitude', 'people_out']]).add_to(seoul_in)

# 지도 확인
seoul_in

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1941553 entries, 0 to 1941552
Data columns (total 4 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   timestamp     object
 1   station_code  int64 
 2   people_in     int64 
 3   people_out    int64 
dtypes: int64(3), object(1)
memory usage: 59.3+ MB


In [18]:
### 출근시간 데이터를 모두 추출

morning_data = data[ (pd.to_datetime(data.timestamp).dt.hour > 5) & 
                                (pd.to_datetime(data.timestamp).dt.hour < 9)]
morning_data

,timestamp,station_code,people_in,people_out
1,2021-01-01T06:00:00.000+09:00,150,111,355
2,2021-01-01T07:00:00.000+09:00,150,157,438
3,2021-01-01T08:00:00.000+09:00,150,306,592
20,2021-01-01T06:00:00.000+09:00,151,45,145
21,2021-01-01T07:00:00.000+09:00,151,75,176
...,...,...,...,...
1941517,2021-12-31T07:00:00.000+09:00,2827,279,120
1941518,2021-12-31T08:00:00.000+09:00,2827,391,311
1941535,2021-12-31T06:00:00.000+09:00,2828,83,41
1941536,2021-12-31T07:00:00.000+09:00,2828,279,119


In [19]:
### 출근 시간 데이터를 역(station_code) 기준으로 그룹화하여 합계 구하기
morning_station_sum = morning_data.groupby("station_code").sum()
morning_station_sum

,people_in,people_out
station_code,,
150,1173032,3383130
151,192838,2694129
152,193164,3595975
153,179542,1182753
154,165788,1733838
...,...,...
2824,1037376,434322
2825,372556,156490
2826,499319,168571


In [20]:
## 출근 시간 데이터와 역 정보 합치기
morning_joined_data = morning_station_sum.join(station_info)
morning_joined_data

,people_in,people_out,geo.latitude,geo.longitude
station_code,,,,
150,1173032,3383130,37.554648,126.972559
151,192838,2694129,37.564718,126.977108
152,193164,3595975,37.570161,126.982923
153,179542,1182753,37.571607,126.991806
154,165788,1733838,37.570926,127.001849
...,...,...,...,...
2824,1037376,434322,37.445210,127.156866
2825,372556,156490,37.440918,127.147564
2826,499319,168571,37.437428,127.140722


In [22]:
## 출근 시간 승차 인원 히트맵
morning_seoul_in = folium.Map(location=[37.55, 126.98], zoom_start = 12) 

HeatMap(data = morning_joined_data[['geo.latitude', 'geo.longitude', 'people_in']]).
                                                              add_to(morning_seoul_in) 
morning_seoul_in

In [24]:
## 출근 시간 하차 인원 히트맵
morning_seoul_out = folium.Map(location=[37.55, 126.98], zoom_start = 12) 

HeatMap(data = morning_joined_data[['geo.latitude', 'geo.longitude', 'people_out']]).
                                                            add_to(morning_seoul_out) 
morning_seoul_out

In [25]:
# 퇴근 시간 데이터를 모두 추출

evening_data = data[ (pd.to_datetime(data.timestamp).dt.hour > 17) & 
                                (pd.to_datetime(data.timestamp).dt.hour < 20)]
evening_data

,timestamp,station_code,people_in,people_out
13,2021-01-01T18:00:00.000+09:00,150,658,437
14,2021-01-01T19:00:00.000+09:00,150,579,425
32,2021-01-01T18:00:00.000+09:00,151,294,121
33,2021-01-01T19:00:00.000+09:00,151,186,57
51,2021-01-01T18:00:00.000+09:00,152,572,274
...,...,...,...,...
1941510,2021-12-31T19:00:00.000+09:00,2826,143,317
1941528,2021-12-31T18:00:00.000+09:00,2827,351,347
1941529,2021-12-31T19:00:00.000+09:00,2827,206,176
1941547,2021-12-31T18:00:00.000+09:00,2828,184,382


In [26]:
# 퇴근 시간 데이터를 역(station_code) 기준으로 그룹화하여 더하기
evening_station_sum = evening_data.groupby("station_code").sum()
evening_station_sum

,people_in,people_out
station_code,,
150,2987183,1343062
151,2034394,269504
152,2940106,609696
153,1688404,518029
154,1592971,357086
...,...,...
2824,361746,726332
2825,137833,329552
2826,135659,313572


In [27]:
## 퇴근 시간 데이터와 역 정보 합치기
evening_joined_data = evening_station_sum.join(station_info)
evening_joined_data

,people_in,people_out,geo.latitude,geo.longitude
station_code,,,,
150,2987183,1343062,37.554648,126.972559
151,2034394,269504,37.564718,126.977108
152,2940106,609696,37.570161,126.982923
153,1688404,518029,37.571607,126.991806
154,1592971,357086,37.570926,127.001849
...,...,...,...,...
2824,361746,726332,37.445210,127.156866
2825,137833,329552,37.440918,127.147564
2826,135659,313572,37.437428,127.140722


In [28]:
## 퇴근 시간 승차 인원 히트맵
evening_seoul_in = folium.Map(location=[37.55, 126.98], zoom_start = 12) 

HeatMap(data = evening_joined_data[['geo.latitude', 'geo.longitude', 'people_in']],
          gradient={0.3: 'blue', 0.6: 'yellow', 1.0: 'red'}).add_to(evening_seoul_in) 
evening_seoul_in

In [29]:
## 퇴근 시간 하차 인원 히트맵
evening_seoul_out = folium.Map(location=[37.55, 126.98], zoom_start = 12) 

HeatMap(data = evening_joined_data[['geo.latitude', 'geo.longitude', 'people_out']],
          gradient={0.3: 'blue', 0.6: 'yellow', 1.0: 'red'}).add_to(evening_seoul_out) 
evening_seoul_out